In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import os
import sys
from ocp_table_tpot.globals import Globals as gd
from tpot import TPOTRegressor
sys.path.insert(0,'..')

from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler,MinMaxScaler,PolynomialFeatures,QuantileTransformer,Normalizer
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from sklearn.metrics import mean_squared_error,make_scorer
from copy import copy
from tpot.builtins import StackingEstimator


from src.models.model import mase,TimeSeriesSplitImproved
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,RANSACRegressor,Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from skgarden.quantile import RandomForestQuantileRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
import umap


df_tsfresh = pd.read_pickle(f'../data/processed/train_test_tsfresh_6.pkl').reset_index(level = 0)
data_dict = pd.read_pickle(f'../data/processed/data_dict_all.pkl')





In [ ]:

year = 2019
tgt = 'rougher.output.recovery'

X = data_dict[year]['X_train_tsclean']
print(X.shape)
y = data_dict[year]['y_train']
X_test = data_dict[year]['X_test_ts']
mask = data_dict[year]['mask']
exclude_pts = data_dict[year]['excl'].set_index('date').tz_localize('UTC')
#mask_na_two_row=y[y[tgt].isna()].index.union(y[y[tgt].isna()].index + pd.Timedelta('1 hour')).union(y[y[tgt].isna()].index + pd.Timedelta('2 hour'))
inds = mask.index.difference(exclude_pts.index)

X=X.loc[inds,:]
y=y.loc[inds,:]
mask=mask[inds]
print(X.shape)


In [ ]:
print(f'1) X shape: {X.shape},y: {y.shape}')
X = X[mask]
y = y[mask][tgt]
print(f'2) Train shape: {X.shape}')
X_filt = X.filter(regex  ="rougher|hour|dayw",axis = 1)
X = X_filt
train_df = pd.concat([X,y],axis= 1)
train_df.head()

In [ ]:
#Fit Isolation RF
from sklearn.ensemble import IsolationForest
irf = IsolationForest(verbose =1,contamination='auto',behaviour='new')

irf.fit(train_df[X.columns])

irf_preds = irf.predict(train_df[X.columns])

(irf_preds > 0).sum()


## Create a function to fit a base model on `K-1` folds, predict on `1` fold

In [ ]:
# This function takes one model and fit it to the train and test data
# It returns the model MASE, CV prediction, and test prediction
def base_fit(model, folds, features, target, trainData, testData):
    # Initialize empty lists and matrix to store data
    model_mase = []
    model_val_predictions = np.empty((trainData.shape[0], 1))
    k=0
    # Loop through the index in KFolds
    model_test_predictions=np.zeros((testData.shape[0],))
    model_val_true=np.zeros((trainData.shape[0],1))

    for train_index, val_index in folds.split(trainData):
        k=k+1
        # Split the train data into train and validation data
        train, validation = trainData.iloc[train_index], trainData.iloc[val_index]
        # Get the features and target
        train_features, train_target = train[features], train[target]
        validation_features, validation_target = validation[features], validation[target]
        
        # Fit the base model to the train data and make prediciton for validation data
        if (model.__class__ == xgb.sklearn.XGBRegressor) | (model.__class__ == lgb.sklearn.LGBMRegressor): 
            print('Fitting a boost model with limited tree rounds')
            evalset = [(validation_features,np.ravel(validation_target))]
            model.fit(train_features, np.ravel(train_target),eval_set =evalset,early_stopping_rounds = 20,verbose = False)
        else:
            model.fit(train_features, train_target.values)
        
        if (model.__class__ == xgb.sklearn.XGBRegressor):
            print(model.best_ntree_limit)
            print('Using xgboost with limited tree rounds')
            validation_predictions = model.predict(validation_features,ntree_limit = model.best_ntree_limit)

        elif (model.__class__ == lgb.sklearn.LGBMRegressor):
            print(model.best_iteration_)
            print('Using lgbmboost with limited tree rounds')
            validation_predictions = model.predict(validation_features,num_iteration = model.best_iteration_)
        else:
            print('Using generic predict')
            validation_predictions = model.predict(validation_features)
        
        
        # Calculate and store the MASE for validation data
        print(mase(validation_predictions,validation_target))
        #model_mase.append(mase(validation_predictions,validation_target))
        
        # Save the validation prediction for level 1 model training
        model_val_predictions[val_index, 0] = validation_predictions.reshape(validation.shape[0])
        model_val_true[val_index,0] = validation_target.values
        model_test_predictions += model.predict(testData[features])    
        
    model_test_predictions = model_test_predictions/k
    # Fit the base model to the whole training data
    #model.fit(trainData[features], np.ravel(trainData[target]))
    # Get base model prediction for the test data
    #model_test_predictions = model.predict(testData[features])
    # Calculate and store the MASE for validation data
        
    #model_val_predictions = model_val_predictions
    model_mase.append(mase(model_val_predictions,model_val_true))
    
    return(model_mase, model_val_predictions, model_test_predictions)

## Create a function to fit a dictionary of models, and get their OOF predictions from the training data

In [ ]:
# Function that takes a dictionary of models and fits it to the data using baseFit
# The results of the models are then aggregated and returned for level 1 model training
def stacks(level0_models, folds, features, target, trainData, testData):
    num_models = len(level0_models.keys()) #Number of models
    
    # Initialize empty lists and matrix
    level0_trainFeatures = np.empty((trainData.shape[0], num_models))
    level0_testFeatures = np.empty((testData.shape[0], num_models))
    
    # Loop through the models
    for i, key in enumerate(level0_models.keys()):
        print('Fitting %s -----------------------' % (key))
        model_mase, val_predictions, test_predictions = base_fit(level0_models[key], folds, features, target, trainData, testData)
        
        # Print the average MASE for the model
        print('%s average MASE: %s' % (key, np.mean(model_mase)))
        print('\n')
        
        # Aggregate the base model validation and test data predictions
        level0_trainFeatures[:, i] = val_predictions.reshape(trainData.shape[0])
        level0_testFeatures[:, i] = test_predictions.reshape(testData.shape[0])
        
    return(level0_trainFeatures, level0_testFeatures)

## Create a function that trains a dictionary of stackers

In [ ]:
# Function that takes a dictionary of classifiers and train them on base model predictions
def stackerTraining(stacker, folds, level0_trainFeatures, level0_testFeatures, trainData,target = None):
    for k in stacker.keys():
        print('Training stacker %s' % (k))
        stacker_model = stacker[k]
        stacker_mase = []
        y_pred = np.zeros_like(trainData[target].values)
        y_true =  np.zeros_like(trainData[target].values)
        for t, v in folds.split(X, y):
            train, validation = level0_trainFeatures[t,:], level0_trainFeatures[v,:]
            # Get the features and target
            train_features, train_target = train, trainData.iloc[t][target]
            validation_features, validation_target = validation, trainData.iloc[v][target]
            
            
            if (stacker_model.__class__ == xgb.sklearn.XGBRegressor) | (stacker_model.__class__ == lgb.sklearn.LGBMRegressor): 
                print('Fitting a boost model with limited tree rounds')
                evalset = [(validation_features,np.ravel(validation_target))]
                stacker_model.fit(train_features, np.ravel(train_target),eval_set =evalset,early_stopping_rounds = 20,verbose = False)
                print(stacker_model.best_iteration_)
            else:
                stacker_model.fit(level0_trainFeatures[t,:], train_target)
                
            y_pred[v] = stacker_model.predict(level0_trainFeatures[v])
            y_true[v] = trainData.iloc[v][target].values
        
        stacker_mase =mase(y_pred,y_true)
        average_mase = mase(level0_trainFeatures.mean(axis=1),y_true)
        print('%s Stacker MASE: %s' % (k, stacker_mase))
        print('%s Averaging MASE: %s' % (k, average_mase))

        
        

In [ ]:
# Get the K fold indexes
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=False, random_state=156)

## Set up the dictionaries of level 0 and level 1 models

In [ ]:
# A dictionary of base models
scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),PCA(whiten=True))
level0_models = {}
# level0_models['Lasso'] = make_pipeline(scaler, Lasso(alpha =0.005, random_state=1,max_iter = 2000))
# level0_models['ElasticNet'] = make_pipeline(scaler,ElasticNet(alpha = 0.001))
# level0_models['XGB_rougher_base_a'] = xgb.XGBRegressor(learning_rate=0.05,
#                             n_estimators=400,**{'max_depth': 7, 'gamma': '44.954', 'colsample_bytree': '0.395', 'subsample': '0.993', 'min_child_weight': '133.132'},
#                             silent=1,
#                              random_state =48, nthread = -1)

# level0_models['XGB_rougher_base_b'] =xgb.XGBRegressor(learning_rate=0.05,
#                             n_estimators=400,**{'max_depth': 7, 'gamma': '44.954', 'colsample_bytree': '0.395', 'subsample': '0.993', 'min_child_weight': '133.132'},
#                             silent=1,
#                              random_state =38, nthread = -1)

# # level0_models['XGB_rougher_base_c'] = xgb.XGBRegressor(learning_rate=0.05,
# #                             n_estimators=400,**{'max_depth': 4, 'gamma': '5.102', 'colsample_bytree': '0.344', 'subsample': '0.884', 'min_child_weight': '9.622'},
# #                             silent=1,
# #                              random_state =82, nthread = -1)

# # level0_models['XGB_rougher_base_d'] = xgb.XGBRegressor(learning_rate=0.05,
# #                             n_estimators=400,**{'max_depth': 5, 'gamma': '3.906', 'colsample_bytree': '0.401', 'subsample': '0.760', 'min_child_weight': '9.563'},
# #                             silent=1,
# #                              random_state =11, nthread = -1)


level0_models['LGBM_rougher_base_a'] = lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=500,random_state=91,
                              **{'max_depth': 5, 'num_leaves': 100, 'feature_fraction': '0.363', 'bagging_fraction': '0.262'})

level0_models['LGBM_rougher_base_b'] =lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=500,random_state=92,
                              **{'max_depth': 4, 'num_leaves': 110, 'feature_fraction': '0.448', 'bagging_fraction': '0.445'})
level0_models['LGBM_rougher_base_c'] =lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=500,random_state=93,
                              **{'max_depth': 4, 'num_leaves': 155, 'feature_fraction': '0.449', 'bagging_fraction': '0.598'})
level0_models['LGBM_rougher_base_d'] =lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=500,random_state=94,
                             **{'max_depth': 5, 'num_leaves': 210, 'feature_fraction': '0.472', 'bagging_fraction': '0.682'})
level0_models['LGBM_rougher_base_e']= lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=500,random_state=7,
                              **{'max_depth': 5, 'num_leaves': 200, 'feature_fraction': '0.45', 'bagging_fraction': '0.72'})
obj = 'mae'
level0_models['LGBM_rougher_base_f']= lgb.LGBMRegressor(objective=obj,
                              learning_rate=0.07, n_estimators=500,random_state=8,
                              **{'max_depth': 4, 'num_leaves': 63, 'feature_fraction': '0.879', 'bagging_fraction': '0.727'})
level0_models['LGBM_rougher_base_g']= lgb.LGBMRegressor(objective=obj,
                              learning_rate=0.07, n_estimators=500,random_state=9,
                              **{'max_depth': 5, 'num_leaves': 65, 'feature_fraction': '0.879', 'bagging_fraction': '0.727'})
level0_models['LGBM_rougher_base_h']= lgb.LGBMRegressor(objective=obj,
                              learning_rate=0.07, n_estimators=500,random_state=10,
                              **{'max_depth': 4, 'num_leaves': 60, 'feature_fraction': '0.797', 'bagging_fraction': '0.982'})
level0_models['LGBM_rougher_base_i'] = lgb.LGBMRegressor(objective=obj,
                              learning_rate=0.07, n_estimators=500,random_state=12,
                              **{'max_depth': 5, 'num_leaves': 60, 'feature_fraction': '0.8', 'bagging_fraction': '0.92'})

# level0_models['KNN_rougher_a'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 254, 'weights': 'distance', 'leaf_size': 16}))
# level0_models['KNN_rougher_b'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 50, 'weights': 'distance', 'leaf_size': 18}))
# level0_models['KNN_rougher_c'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 15, 'weights': 'distance', 'leaf_size': 30.0}))
# level0_models['KNN_rougher_d'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 5, 'weights': 'uniform', 'leaf_size': 24.0}))
# level0_models['KNN_rougher_b_bray'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 50, 'weights': 'distance','metric':'braycurtis', 'leaf_size': 18}))
# level0_models['KNN_rougher_c_bray'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 15, 'weights': 'distance', 'leaf_size': 30.0,'metric':'braycurtis'}))
# level0_models['KNN_rougher_d_bray'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 5, 'weights': 'uniform', 'leaf_size': 24.0,'metric':'braycurtis'}))




In [ ]:
# Train all the base models in the dictionary
level0_trainFeatures_rougher, level0_testFeatures_rougher = stacks(level0_models, kf, X.columns, tgt, train_df, X_test)


In [ ]:
# A dictionary of level 1 model to train on base model predictions
stacker = {
            'Enet': ElasticNet(alpha = 0.001),
#           'BR':BayesianRidge(n_iter = int(3e3)),
#           'Ridge1':Ridge(),
            'Ridge50':Ridge(alpha=60),
           'Ridge1e2':Ridge(alpha=1e2),
           'Ridge5e2':Ridge(alpha=5e2),
           'Ridge1e3':Ridge(alpha=1e3),
           'Ridge5e3':Ridge(alpha=5e3),
          'Lasso': Lasso(alpha =0.005, random_state=1,max_iter = 2000),
      'positive_Lasso': Lasso(alpha=0.0001,precompute=True,max_iter=1000, positive=True, random_state=9999, selection='random'),
          'LGBM': lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.07, n_estimators=500,random_state=38,
                              **{'max_depth': 2, 'num_leaves': 8, 'feature_fraction': '0.8', 'bagging_fraction': '0.8'})}

stackerTraining(stacker, kf, level0_trainFeatures_rougher, level0_testFeatures_rougher, train_df,target = tgt)
level1_model = stacker['LGBM'].fit(level0_trainFeatures_rougher,train_df[tgt])
level1_test_pred_rougher = level1_model.predict(level0_testFeatures_rougher)
rougher_meta_train = level0_trainFeatures_rougher
rougher_meta_test = level0_testFeatures_rougher

level1_test_pred_rougher.shape

# Let's try to figure out the stacking Level 2

In [ ]:
def solve_pg(A, b, momentum=0.9, maxiter=1000):
    from cvxpy import Variable,Parameter,Minimize,square,norm,Problem
    M, N = A.shape
    x = np.zeros(N)

    AtA = A.T.dot(A)
    Atb = A.T.dot(b)

    stop_count = 0

    # projection helper
    x_ = Variable(N)
    v_ = Parameter(N)
    objective_ =  Minimize(0.5 * norm(x_ - v_, 1))
    constraints_ = [sum(x_) == 1]
    problem_ = Problem(objective_, constraints_)

    def gradient(x):
        return AtA.dot(x) - Atb

    def obj(x):
        return 0.5 * np.linalg.norm(A.dot(x) - b,ord=1)

    it = 0
    while True:
        grad = gradient(x)

        # line search
        alpha = 1
        beta = 0.5
        sigma=1e-2
        old_obj = obj(x)
        while True:
            new_x = x - alpha * grad
            new_obj = obj(new_x)
            if old_obj - new_obj >= sigma * grad.dot(x - new_x):
                break
            else:
                alpha *= beta

        x_old = x[:]
        x = x - alpha*grad

        # projection
        v_.value = x
        problem_.solve()
        x = np.array(x_.value.flat)

        y = x + momentum * (x - x_old)

        if np.abs(old_obj - obj(x)) < 1e-2:
            stop_count += 1
        else:
            stop_count = 0

        if stop_count == 3:
            print('early-stopping @ it: ', it)
            return x

        it += 1

        if it == maxiter:
            return x

x = solve_pg(level0_trainFeatures_rougher, train_df[tgt].values)
print('sum x: ', np.sum(x))
print(mase(level0_trainFeatures_rougher.dot(x),train_df[tgt].values))


# Repeat the procedure with Final

In [ ]:
year = 2019
tgt = 'final.output.recovery'

X = data_dict[year]['X_train_tsclean']
y = data_dict[year]['y_train_tsclean']
X_test = data_dict[year]['X_test_ts']

mask = data_dict[year]['mask']
exclude_pts = data_dict[year]['excl'].set_index('date').tz_localize('UTC')
#mask_na_two_row=y[y[tgt].isna()].index.union(y[y[tgt].isna()].index + pd.Timedelta('1 hour')).union(y[y[tgt].isna()].index + pd.Timedelta('2 hour'))
inds = mask.index.difference(exclude_pts.index)
print(X.shape)

X=X.loc[inds,:]
y=y.loc[inds,:]
mask=mask[inds]
print(X.shape)

print(f'1) X shape: {X.shape},y: {y.shape}')
X = X[mask]
y = y[mask][tgt]

train_df = pd.concat([X,y],axis= 1)
print(f'1) X shape: {X.shape},y: {y.shape}')

train_df.head()

## Create dicts of stackers and base models

In [ ]:
# A dictionary of base models
scaler = make_pipeline(QuantileTransformer(output_distribution='normal'),PCA(whiten=True))
level0_models = {}
# level0_models['Lasso'] = make_pipeline(scaler, Lasso(alpha =0.005, random_state=1,max_iter = 2000))
# level0_models['ElasticNet'] = make_pipeline(scaler,ElasticNet(alpha = 0.001))
def fair_obj(preds, dtrain):
    """y = c * abs(x) - c**2 * np.log(abs(x)/c + 1)"""
    x = preds - dtrain.get_labels()
    c = 1
    den = abs(x) + c
    grad = c*x / den
    hess = c*c / den ** 2
    return grad, hess

# level0_models['XGB_final_base_a'] = xgb.XGBRegressor(learning_rate=0.05,
#                             n_estimators=400,**{'max_depth': 3, 'gamma': '17.158', 'colsample_bytree': '0.442', 'subsample': '0.644', 'min_child_weight': '9.733'},
#                             silent=1,
#                              random_state =48, nthread = -1)

# level0_models['XGB_final_base_b'] =xgb.XGBRegressor(learning_rate=0.05,
#                             n_estimators=400,**{'max_depth': 4, 'gamma': '18.571', 'colsample_bytree': '0.745', 'subsample': '0.681', 'min_child_weight': '9.024'},
#                             silent=1,
#                              random_state =38, nthread = -1)

# level0_models['XGB_final_base_c'] = xgb.XGBRegressor(learning_rate=0.05,
#                             n_estimators=400,**{'max_depth': 2, 'gamma': '16.491', 'colsample_bytree': '0.522', 'subsample': '0.844', 'min_child_weight': '5.096'},
#                             silent=1,
#                              random_state =82, nthread = -1)

# level0_models['XGB_final_base_d'] = xgb.XGBRegressor(learning_rate=0.05,
#                             n_estimators=400,**{'max_depth': 3, 'gamma': '16.766', 'colsample_bytree': '0.540', 'subsample': '0.774', 'min_child_weight': '12.674'},
#                             silent=1,
#                              random_state =11, nthread = -1)

level0_models['LGBM_final_base_a'] = lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=400,random_state=96,
                              **{'max_depth': 6, 'num_leaves': 10, 'feature_fraction': '0.411', 'bagging_fraction': '0.827'})

level0_models['LGBM_final_base_b'] =lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=400,random_state=973,
                              **{'max_depth': 6, 'num_leaves': 15, 'feature_fraction': '0.515', 'bagging_fraction': '0.469'})

level0_models['LGBM_final_base_c'] =lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=400,random_state=937,
                              **{'max_depth': 3, 'num_leaves': 195, 'feature_fraction': '0.635', 'bagging_fraction': '0.673'})
level0_models['LGBM_final_base_d'] =lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.05, n_estimators=400,random_state=49,
                              **{'max_depth': 4, 'num_leaves': 70, 'feature_fraction': '0.795', 'bagging_fraction': '0.656'})



# level0_models['LGBM_final_base_e'] = lgb.LGBMRegressor(objective=obj,
#                               learning_rate=0.07, n_estimators=500,random_state=7,
#                               **{'max_depth': 4, 'num_leaves': 63, 'feature_fraction': '0.89', 'bagging_fraction': '0.757'})
# level0_models['LGBM_final_base_f'] = lgb.LGBMRegressor(objective=obj,
#                               learning_rate=0.07, n_estimators=500,random_state=8,
#                               **{'max_depth': 4, 'num_leaves': 63, 'feature_fraction': '0.879', 'bagging_fraction': '0.727'})
# level0_models['LGBM_final_base_g'] = lgb.LGBMRegressor(objective=obj,
#                               learning_rate=0.07, n_estimators=500,random_state=9,
#                               **{'max_depth': 5, 'num_leaves': 65, 'feature_fraction': '0.879', 'bagging_fraction': '0.727'})
# level0_models['LGBM_final_base_h'] = lgb.LGBMRegressor(objective=obj,
#                               learning_rate=0.07, n_estimators=500,random_state=10,
#                               **{'max_depth': 4, 'num_leaves': 60, 'feature_fraction': '0.797', 'bagging_fraction': '0.982'})
# level0_models['LGBM_final_base_i'] = lgb.LGBMRegressor(objective=obj,
#                               learning_rate=0.07, n_estimators=500,random_state=12,
#                               **{'max_depth': 5, 'num_leaves': 60, 'feature_fraction': '0.8', 'bagging_fraction': '0.92'})



# #level0_models['KNN_final_a'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 254, 'weights': 'distance', 'leaf_size': 16}))
# level0_models['KNN_final_b'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 50, 'weights': 'distance', 'leaf_size': 18}))
# level0_models['KNN_final_c'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 15, 'weights': 'distance', 'leaf_size': 30.0}))
# level0_models['KNN_final_d'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 5, 'weights': 'uniform', 'leaf_size': 24.0}))
# level0_models['KNN_rougher_b_bray'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 50, 'weights': 'distance','metric':'braycurtis', 'leaf_size': 18}))
# level0_models['KNN_rougher_c_bray'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 15, 'weights': 'distance', 'leaf_size': 30.0,'metric':'braycurtis'}))
# level0_models['KNN_rougher_d_bray'] = make_pipeline(scaler,KNeighborsRegressor(n_jobs = -1,**{'n_neighbors': 5, 'weights': 'uniform', 'leaf_size': 24.0,'metric':'braycurtis'}))




In [ ]:
# Train all the base models in the dictionary
level0_trainFeatures_final, level0_testFeatures_final = stacks(level0_models, kf, X.columns, tgt, train_df, X_test)

In [ ]:
# A dictionary of level 1 model to train on base model predictions
stacker = {
           'Enet': ElasticNet(alpha = 0.001),
          #'RF':RandomForestRegressor(max_depth = 3,n_estimators=200),
          'BR':BayesianRidge(n_iter = int(3e3)),
          'Ridge1':Ridge(),
          'Ridge1e2':Ridge(alpha=1e2),
          'Ridge5e2':Ridge(alpha=5e2),
          'Ridge1e3':Ridge(alpha=1e3),
          'Ridge5e3':Ridge(alpha=5e3),
          'Lasso': Lasso(alpha =0.005, random_state=1,max_iter = 2000),
          'LGBM': lgb.LGBMRegressor(objective='mae',
                              learning_rate=0.07, n_estimators=500,random_state=40,
                              **{'max_depth': 3, 'num_leaves':8, 'feature_fraction': '0.9', 'bagging_fraction': '0.7'})}

stackerTraining(stacker, kf, level0_trainFeatures_final, level0_testFeatures_final, train_df,target = tgt)
level1_model = stacker['LGBM'].fit(level0_trainFeatures_final,train_df[tgt])
level1_test_pred_final = level1_model.predict(level0_testFeatures_final)
level1_test_pred_final.shape

In [ ]:
x

## Make a submission:

In [ ]:
preds = pd.DataFrame(data = {'date':X_test.index,'rougher.output.recovery':level1_test_pred_rougher, 'final.output.recovery':level1_test_pred_final}).set_index('date')

stacked_preds_sub = preds
stacked_preds_sub = stacked_preds_sub.reset_index()
stacked_preds_sub['date'] = stacked_preds_sub['date'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
stacked_preds_sub.set_index('date',inplace=True)
stacked_preds_sub.drop_duplicates(inplace=True)
stacked_preds_sub.to_csv('../results/stacked_sub_lgb_lasso_base_alldata_tsclean_fixed_lgbm_lagdiff.csv')

In [ ]:
preds.plot(style=['o','o'],figsize = (20,10),alpha=0.9)


In [ ]:
preds_av = pd.DataFrame(data = {'date':X_test.index,'rougher.output.recovery':level0_testFeatures_rougher.mean(axis=1), 'final.output.recovery':level0_testFeatures_final.mean(axis=1)})

preds_av['date'] = preds_av['date'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
preds_av.set_index('date',inplace=True)
preds_av.plot(figsize = (20,10),style=['o','o'],alpha=0.9)

preds_av.to_csv('../results/stacked_sub_lgb_lasso_base_alldata_tsclean_r-filt-f-meta.fixed_best_mod_averaged_lagdiff.csv')

In [ ]:
r = np.power(level0_testFeatures_rougher.prod(axis=1),1/level0_testFeatures_rougher.shape[1])
f = np.power(level0_testFeatures_final.prod(axis=1),1/level0_testFeatures_final.shape[1])

preds_av = pd.DataFrame(data = {'date':X_test.index,'rougher.output.recovery':r, 'final.output.recovery':f})
preds_av['date'] = preds_av['date'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
preds_av.set_index('date',inplace=True)
preds_av.plot(figsize = (20,10),style=['o','o'],alpha=0.9)